#### For every routine
#### Sample times per activity
#### Check for overlaps and get actions in order
#### Create single script with times
#### Execute to get timestamped graphs ***

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import os
import shutil
import numpy as np
from random import random

sys.path.append('simulation')
from evolving_graph.scripts import Script, parse_script_line
import evolving_graph.utils as utils
from evolving_graph.scripts import read_script, read_script_from_string, read_script_from_list_string, ScriptParseException, Action
from evolving_graph.execution import ScriptExecutor
from evolving_graph.environment import EnvironmentGraph

from GraphReader import GraphReader, init_graph_file, scene_num
from ProgramExecutor import read_program

In [ ]:
def time_internal(mins, hrs, day):
    return int(round(((((day)*24)+hrs)*60+mins)/DT))

def day_num(day_of_week):
    return {'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6}[day_of_week]

In [ ]:
init_graph = GraphReader(graph_file=init_graph_file)
print(f'Using scene {int(scene_num)-1}, i.e. \'TestScene{scene_num}\'')

In [ ]:
DT = 1   # minutes
NUM_REPEAT_ROUTINES = 5
WEEKEND_DAYS = [day_num(day) for day in ['Saturday','Sunday']]
INTERLEAVING = True
START_TIME = (5,50)
END_TIME = (24,00)

DATASET_DIR = 'sourcedRoutines/trial'
if os.path.exists(DATASET_DIR):
    shutil.rmtree(DATASET_DIR)
os.makedirs(DATASET_DIR)

In [ ]:
class Activity():
    def __init__(self, name, time, script_file=None):
        self.name = name
        start_time = time[0][0]*60 + time[0][1]
        end_time = time[1][0]*60 + time[1][1]
        directory = os.path.join('sourcedScripts', name)
        if script_file is None:
            script_file = np.random.choice(os.listdir(directory))
            headers, self.scripts = read_program(os.path.join(directory,script_file), init_graph.node_map)
        def sample_duration(header):
            durations = (header).split('-')
            assert len(durations)==2, f"Invalid comment {header} in {name}->{script_file}"
            duration_min = int(durations[0].strip())
            duration_max = int(durations[1].strip())
            duration_sampled = random() * (duration_max-duration_min) + duration_min
            return duration_sampled
        self.durations = [sample_duration(header) for header in headers]
        def valid_times(times):
            for time, next_time, duration in zip(times[:-1], times[1:], self.durations[:-1]):
                if next_time-time < duration:
                    return False
            return True
        valid = False
        while not valid:
            times = (np.random.rand(len(headers)) * (end_time-start_time) + start_time).round().astype(int)
            times.sort()
            valid = valid_times(times)
        self.times = times
    
    def get_action_info(self):
        detailed_actions = []
        for t,d,scr in zip(self.times, self.durations, self.scripts):
            t2 = int(round(t+d))
            detailed_actions.append({'time_from':t, 'time_to':t2, 'name':self.name, 'script':scr, 'time_from_h':'{:02d}:{:02d}'.format(t//60, t%60), 'time_to_h':'{:02d}:{:02d}'.format(t2//60, t2%60)})
        return detailed_actions

class Schedule():
    def __init__(self, schedule_file_path=None):
        if schedule_file_path is None:
            schedule_options = []
            for (root,_,files) in os.walk('sourcedSchedules'):
                schedule_options += [os.path.join(root,f) for f in files]
            schedule_file_path = np.random.choice(schedule_options)
        with open(schedule_file_path) as f:
            schedule = json.load(f)
        self.activities = [Activity(act_name, act_time) for act_name,act_time in schedule.items()]
    
    def get_combined_script(self, verbose=False):
        all_actions = []
        for act in self.activities:
            all_actions += act.get_action_info()
        all_actions.sort(key = lambda x : x['time_from'])
        if verbose:
            for a in all_actions:
                print (a['time_from_h']+' to '+a['time_to_h']+' : '+a['name'])
                for l in a['script']:
                    print(' - ',l)
        valid=True
        for end_time, next_start in zip([a['time_to'] for a in all_actions[:-1]], [a['time_from'] for a in all_actions[1:]]):
            if end_time>next_start:
                valid = False
                if verbose:
                    print(f'End time {end_time} of an activity, exceeds start time {next_start} of next activity')
                break
        return all_actions, valid

def get_graphs(all_actions):
    with open (init_graph_file,'r') as f:
        init_graph = EnvironmentGraph(json.load(f))
    name_equivalence = utils.load_name_equivalence()
    executor = ScriptExecutor(init_graph, name_equivalence)
    graphs = []
    for action in all_actions:
        print('Executing : ')
        for l in action['script']:
            print(' - ',str(l))
        success, state, _ = executor.execute(Script(action['script']), w_graph_list=False)
        if not success:
            raise RuntimeError('Execution of {} starting at {} failed'.format(action['name'], action['time_from_h']))
        graphs.append(state._graph)
    return graphs



In [ ]:
s = Schedule()
actions, valid = s.get_combined_script()
graphs = get_graphs(actions)